In [1]:
import pandas as pd
import pickle
import xgboost as xgb
import nfl_data_py as nfl
import plotly.express as px

In [2]:
overUnderFifteenModel = pickle.load(open('overUnderFifteen.pkl', 'rb'))
overUnderEighteenModel = pickle.load(open('overUnderEighteen.pkl', 'rb'))
overUnderTwentyOneModel = pickle.load(open('overUnderTwentyOne.pkl', 'rb'))
overUnderTwentyFourModel = pickle.load(open('overUnderTwentyFour.pkl', 'rb'))

In [3]:
qbData = pd.read_parquet('qb_data_complete.parquet')

In [4]:
curData = qbData[qbData.season == 2021].copy()

In [5]:
remove = ['espn_id', 'season', 'fantasy_points_per_game_next_year', 'fantasy_points_next_year', 'qualified_n-1', 'qualified_n-2', 'qualified']
curFeatures = curData.drop(remove, axis=1).copy()
curData['over_fifteen'] = overUnderFifteenModel.predict(curFeatures)
curData['over_eighteen'] = overUnderEighteenModel.predict(curFeatures)
curData['over_twenty_one'] = overUnderTwentyOneModel.predict(curFeatures)
curData['over_twenty_four'] = overUnderTwentyFourModel.predict(curFeatures)

In [6]:
curData.head()

,espn_id,season,age,height,weight,draft_pick,fantasy_points,games,ppr_sh,completions,...,rushing_yards_per_game_n-2,rushing_tds_per_game_n-2,rushing_fumbles_per_game_n-2,rushing_fumbles_lost_per_game_n-2,sack_fumbles_per_game_n-2,sack_fumbles_lost_per_game_n-2,over_fifteen,over_eighteen,over_twenty_one,over_twenty_four
0,8439.0,2021.0,38.0,74.0,225.0,24.0,332.30,16.0,0.207297,366.0,...,11.4375,0.0625,0.0625,0.0625,0.1875,0.1875,1,1,0,0
1,2330.0,2021.0,44.0,76.0,225.0,33.0,374.74,17.0,0.195705,485.0,...,2.1250,0.1875,0.0000,0.0000,0.1875,0.0625,1,0,0,0
2,4038941.0,2021.0,23.0,78.0,237.0,6.0,380.76,17.0,0.210789,443.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0
3,12483.0,2021.0,33.0,75.0,220.0,1.0,329.74,17.0,0.193814,404.0,...,8.2500,0.0000,0.2500,0.1250,0.3750,0.2500,1,0,0,0
4,3918298.0,2021.0,25.0,77.0,237.0,7.0,402.58,17.0,0.235910,409.0,...,31.8750,0.5625,0.4375,0.1250,0.4375,0.1250,1,1,1,0


In [7]:
range = []
for row in curData.itertuples():
    if row.over_twenty_four == 1:
        range.append('24<')
    elif row.over_twenty_one == 1:
        range.append('21-24')
    elif row.over_eighteen == 1:
        range.append('18-21')
    elif row.over_fifteen == 1:
        range.append('15-18')
    else:
        range.append('<15')

In [8]:
curData['range'] = range

In [9]:
idData = nfl.import_ids()
idSubset = idData[['gsis_id', 'espn_id', 'name', 'team', 'position']]

In [10]:
mergedData = curData.merge(idSubset, on='espn_id')

In [13]:
mergedData[['gsis_id', 'name', 'team', 'position', 'range', 'age', 'fantasy_points_per_game']].to_parquet('qb_predictions.parquet')

In [14]:
mergedData.columns

Index(['espn_id', 'season', 'age', 'height', 'weight', 'draft_pick',
       'fantasy_points', 'games', 'ppr_sh', 'completions',
       ...
       'sack_fumbles_lost_per_game_n-2', 'over_fifteen', 'over_eighteen',
       'over_twenty_one', 'over_twenty_four', 'range', 'gsis_id', 'name',
       'team', 'position'],
      dtype='object', length=164)